# 🚀 Demo: Abstractive Health API Integration

Welcome to the Abstractive Health API demo! This notebook demonstrates a complete workflow for accessing and processing medical records through our API. The workflow is divided into three main parts:

## Step 1: Authentication 🔑
- Obtain your access token to interact with the API

## Step 2: Data Aggregation 📊
You can aggregate patient data in three ways:

1. **Patient Search** 🔍
   - Search the Carequality® network using patient demographics
   - Retrieve clinical documents from connected healthcare providers

2. **Document Upload** 📤
   - Submit your own documents (CDA, FHIR, PDFs, images)
   - Process documents you already have

3. **Hybrid Approach** 🔄
   - First search for patient records in the network
   - Then add your own documents to the same conversation
   - Combine network-retrieved and uploaded documents

## Step 3: Summarization 🧠
- Request an AI-generated clinical summary from the aggregated documents
- Retrieve the structured JSON output
- Optionally push back treatment notes to the network

## Step 4: Push Medical Document 📝
- Submit an updated treatment note for HIE reciprocity

---

# How to use this notebook

- Run each cell sequentially
- Replace placeholders (e.g., tokens, patient metadata) with your test or real data
- Use the demo/test flag where appropriate to avoid affecting production
- Explore the output: summary sections, structured entities (ICD‑10/CPT/HCC), and URLs to documents

Let's get started! 💡


# Step 1: Authentication 🔑

This section demonstrates how to authenticate with the Abstractive Health API and obtain your access token. This token will be used for all subsequent API calls.

In [3]:
import json
import time
import base64
import requests

In [1]:
# Fill in your credentials here
user_api_email = ""  # Your API email
username_api = ""    # Your username
user_api_password = ""  # Your API password

# Authentication and Token Retrieval
token_get_input = {
    'user_api_email': user_api_email,
    'username_api': username_api,
    'user_api_password': user_api_password, 
    'token': ""
}

base_url = "https://api.abstractive.ai/"  # Production URL

In [ ]:
request_url = f"{base_url}get-token"
exchange0_output = requests.post(request_url, json=token_get_input)
print(exchange0_output.content)

In [ ]:
access_token = json.loads(exchange0_output.content.decode('utf-8'))['access_token']
print(access_token)

# Step 2: Data Aggregation 📊

## Option A: Only Patient Search 🔍

This section demonstrates how to search for patient records in the Carequality® network using patient demographics. This is the first method of data aggregation.

### Searching for the Patient

In [6]:
# Define the API endpoint for search-patient
search_patient_url = f"{base_url}search-patient"

# Define patient metadata
patient_metadata = [
    {
        "demographics": {
            "given_name": "",  # Patient's first name
            "family_name": "", # Patient's last name
            "birth_time": "",  # Format: YYYYMMDD
            "administrative_gender_code": "",  # "M" or "F"
            "phone_number": "", # Optional. Format: 123-456-7890
        },
        "addresses": [
            {
                "street_address_line": "", # 123 Main St
                "city": "", # Anytown
                "state": "", # CA
                "postal_code": "", # 12345
                "country": "" # USA
            }
        ]  # Optional: Add address information if needed
    },
]

# Create the search input
search_input = {
    "user_api_email": user_api_email,
    "token": access_token,
    "patient_metadata": patient_metadata,
    "robustness": "100", # Range: 0-100. Higher values produces more documents but slower. Default: 20.
}

# Set headers for the API request
headers = {
    "Content-Type": "application/json"
}

In [ ]:
# Metadata Search
print("Testing search-patient with patient metadata:")
search_response = requests.post(search_patient_url, headers=headers, data=json.dumps(search_input))
print(f"Status Code: {search_response.status_code}")
print(f"Response: {search_response.json()}")

print("\n" + "-"*50 + "\n")

# Validate responses
if search_response.status_code in [200, 202]:
    print("\nMetadata search successful!")
    # Store the conversation_id and pid for future use
    search_result = search_response.json()
    print(f"Conversation ID: {search_result.get('conversation_id')}")
    print(f"Patient ID: {search_result.get('pid')}")
    conversation_id = search_result.get('conversation_id')
    patient_id = search_result.get('pid')
else:
    print(f"\nMetadata search failed with status code: {search_response.status_code}")


### Document Retrieval
This step retrieves the documents found during the patient search. This is part of the patient search workflow.

Upon completed patient search, you will receive a presigned url to a zip folder with all the found patient documents

In [13]:
request_url = f"{base_url}retrieve-patient-docs"

document_retrieval_input = {
    "user_api_email": user_api_email,
    "token": access_token,
    "conversation_id": conversation_id,
    "patient_id": patient_id,
}

In [ ]:
# Test the retrieve-patient-docs API endpoint
headers = {
    "Content-Type": "application/json"
}

# Conversation ID and Patient ID
print("Testing retrieve docs with conversation_id and patient_id:")

while True:
    response = requests.post(request_url, headers=headers, data=json.dumps(document_retrieval_input))
    print(f"Status Code: {response.status_code}")
    response_json = response.json()
    print(f"Response: {response_json}")

    if response.status_code == 200:
        print("\nConversation ID search successful!")
        # You can add more validation here based on expected response structure
        break
    elif response.status_code == 202:
        print(f"Response: {response_json['message']}")
        print("\nProcessing in progress, waiting 30 seconds before retrying...")
        time.sleep(30)
    else:
        print(f"\nConversation ID search failed with status code: {response.status_code}")
        break


## Option B: Only Document Upload 📤

This section demonstrates the second method of data aggregation - uploading your own documents. You can submit CDA, FHIR, PDFs, or images for processing.

In [9]:
# Define the upload endpoint
request_url = f"{base_url}upload-patient-docs"

# Example of how to load and encode your files
# Uncomment and modify these lines with your actual file paths
# pdf1 = base64.b64encode(open("path/to/your/file1.pdf", "rb").read()).decode('utf-8')
# pdf2 = base64.b64encode(open("path/to/your/file2.pdf", "rb").read()).decode('utf-8')
# cda = base64.b64encode(open("path/to/your/cda_file.txt", "rb").read()).decode('utf-8')
# fhir = base64.b64encode(open("path/to/your/fhir_file.json", "rb").read()).decode('utf-8')

# Structure your files dictionary
files = {
    "non_text": [
        # Add your base64 encoded PDFs, PNGs, TIFFs, or JPEGs here
    ],
    "cda": [
        # Add your base64 encoded CDA documents here
    ],
    "fhir": [
        # Add your base64 encoded FHIR documents here
    ],
}

upload_input = {
    "user_api_email": user_api_email,
    "patient_metadata": patient_metadata[0], # Dictionary of single patient metadata
    "token": access_token,
    "files": files
}

In [ ]:
# Make the API requests
headers = {
    "Content-Type": "application/json"
}

print("\n" + "-"*50 + "\n")
print("Testing upload with patient_metadata:")
response = requests.post(request_url, headers=headers, data=json.dumps(upload_input))
print(f"Status Code: {response.status_code}")
print(f"Response: {response.json()}")


## Option C: Hybrid Approach 🔄

This section demonstrates how to combine network-retrieved documents with your own uploaded documents in the same conversation. This is useful when you want to supplement network data with your own records.

To use the hybrid approach:
1. First perform a patient search (Option A)
2. Then use the same conversation_id when uploading your documents
3. This will ensure all documents are processed together


In [ ]:
# Example of hybrid approach - using the same conversation_id from patient search
hybrid_upload_input = {
    "user_api_email": user_api_email,
    "patient_metadata": patient_metadata[0],
    "token": access_token,
    "files": files,
    "conversation_id": conversation_id  # Using the conversation_id from patient search
}


In [ ]:
# Make the API request for hybrid upload
hybrid_response = requests.post(
    f"{base_url}upload-patient-docs", 
    headers=headers, 
    data=json.dumps(hybrid_upload_input)
)
print(f"Status Code: {hybrid_response.status_code}")
print(f"Response: {hybrid_response.json()}")


# Step 3: Summarization 🧠

This section demonstrates how to:
1. Request an AI-generated clinical summary
2. Retrieve the structured JSON output
3. Analyze the summarized data

The summary will include:
- Patient history and presenting illness (HPI)
- Lab results and imaging studies
- Vital signs
- Allergies
- Family and social history
- Past clinical events
- Historical followups
- Provider information
- Medication history


## Request an AI-generated clinical summary

In [ ]:
# Define the request-summary API endpoint
request_summary_url = f"{base_url}/request-summary"

# Create the request body
request_summary_input = {
    "user_api_email": user_api_email,
    "patient_id": patient_id,
    "conversation_id": conversation_id,
    "token": access_token
}

# Test the request-summary API endpoint
print("Testing request-summary API:")
request_summary_response = requests.post(
    request_summary_url, 
    headers={"Content-Type": "application/json"}, 
    data=json.dumps(request_summary_input)
)
print(f"Status Code: {request_summary_response.status_code}")
print(f"Response: {request_summary_response.json()}")

# Validate response
if request_summary_response.status_code == 202:
    print("\nSummary request accepted and being processed!")
    summary_result = request_summary_response.json()
else:
    print(f"\nSummary request failed with status code: {request_summary_response.status_code}")
    print(f"Failure reason: {request_summary_response.json().get('message')}")


## Retrieve AI Summary

In [ ]:
# Define the retrieve-summary API endpoint
retrieve_summary_url = f"{base_url}/retrieve-summary"

# Create the request body using the same conversation_id
retrieve_summary_input = {
    "conversation_id": conversation_id,
    "token": access_token,
    "user_api_email": user_api_email,
    "patient_id": patient_id
}

# Test the retrieve-summary API endpoint
print("Testing retrieve-summary API:")
while True:
    retrieve_summary_response = requests.post(
        retrieve_summary_url, 
        headers=headers, 
        data=json.dumps(retrieve_summary_input)
    )
    print(f"Status Code: {retrieve_summary_response.status_code}")
    response_json = retrieve_summary_response.json()
    print(f"Response: {response_json}")

    # Validate response
    if retrieve_summary_response.status_code == 200:
        print("\nSummary retrieved successfully!")
        print(f"Posted: {response_json.get('posted')}")
        print(f"URL: {response_json.get('url')}")
        break
    elif retrieve_summary_response.status_code == 202:
        print("\nSummary is still being processed. Retrying in 30 seconds...")
        time.sleep(30)
    else:
        print(f"\nSummary retrieval failed with status code: {retrieve_summary_response.status_code}")
        print(f"Failure reason: {retrieve_summary_response.json().get('failure_reason')}")
        break


In [ ]:
# Once the summary is ready, you can download it using the presigned URL
output_json = requests.get(retrieve_summary_response.json()['url']).content

# The output will be a JSON object containing structured clinical data
# Example structure (actual data will be populated with patient information):
example_structure = {
    "HPI": "Patient history and presenting illness",
    "Labs": ["Lab test 1: result", "Lab test 2: result"],
    "Images": ["Imaging study 1: findings", "Imaging study 2: findings"],
    "Vitals": ["BP: value", "HR: value", "Temperature: value"],
    "Allergies": ["Allergy 1: reaction", "Allergy 2: reaction"],
    "Family History": "Relevant family medical history",
    "Social History": "Relevant social and lifestyle factors",
    "Past Clinical Events": ["Event 1", "Event 2"],
    "All Historical Followups": ["Followup 1", "Followup 2"],
    "All Historical Providers": ["Provider 1", "Provider 2"],
    "All Historical Medications": ["Medication 1", "Medication 2"]
}

print("Retrieved summary structure:")
print(json.dumps(example_structure, indent=2))

# Step 4: Push Medical Document 📝

This final step demonstrates how to submit an updated treatment note back to the Carequality® network, fulfilling HIE reciprocity requirements. This allows you to:
1. Share your findings with other healthcare providers
2. Maintain continuity of care
3. Contribute to the patient's comprehensive health record

The document you push should include:
- Patient information
- Provider/Author details
- Clinical assessment
- Treatment summary
- Any relevant updates to the patient's record


In [ ]:
push_document_url = f"{base_url}push-document"

# Prepare the input for pushing a document
# This includes patient details, author information, and summary content
push_document_input = {
    # Patient information
    "patient": {
        "pid": "unique id of the patient",                   # Required - Unique patient identifier
        "given_name": "first name of the patient",           # Required if no pid provided
        "family_name": "last name of the patient",           # Required if no pid provided  
        "birth_time": "date of birth of the patient",        # Required if no pid provided
        "administrative_gender_code": "sex of the patient",   # Required if no pid provided
        "phone_number": "phone number of the patient",       # Optional contact info
        "email": "email of the patient",                     # Optional contact info
        "street_address_line": "patient's address line",     # Optional address info
        "city": "patient's city",                           # Optional address info
        "state": "patient's state",                         # Optional address info
        "postal_code": "patient's zipcode",                 # Optional address info
        "country": "patient's country"                      # Optional address info
    },
    
    # Author/Provider information 
    "author": {
        "given_name": "first name of the author",           # Required
        "family_name": "last name of the author",           # Required
        "authored_on": "date of the document",              # Required - Document creation date
        "npi": "national provider identifier",              # Required - Provider NPI number
        "specialty": "provider's specialty"                 # Required - Medical specialty
    },
    
    # Document content and metadata
    "summary_assessment": "clinical context of the summary in plain text",  # Required
    "summary_edited": "summary with edits made",                           # Required
    "summary_format": "text/json. Defaults to text.",                      # Optional - Format specification
    "user_api_email": "string",                                           # Required - API user email
    "token": "string"                                                     # Required - Authentication token
}

In [ ]:
push_document_response = requests.post(push_document_url, headers=headers, data=json.dumps(push_document_input))
print(f"Status Code: {push_document_response.status_code}")
print(f"Response: {push_document_response.json()}")

if push_document_response.status_code == 200:
    print("\nDocument pushed successfully!")
    push_document_result = push_document_response.json()
    print(f"Conversation ID: {push_document_result.get('conversation_id')}")